<a href="https://colab.research.google.com/github/fauzinurfalah/SentimentIMDB/blob/main/KEL3_DATA_MINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kelompok 3 Data Mining
1. Ahda Faizalziddi Al Aziz     (STI202303600)
2. Ilham Aufa Nugroho           (STI202303346)
3. Fauzi Nur Falah              (STI202303409)
4. Muhammad Syarif Hidayatullah (STI202303527)
5. Tri Wahyu Hidayat            (STI202303353)
6. Muhammad Roy Setiawan        (STI202303331)
7. Sahrur Anam                  (STI202303552)
8. Muh Januar Imam Saputra      (STI202202623)

In [1]:
# Instal library
!pip install pandas numpy scikit-learn seaborn matplotlib nltk wordcloud imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud

# ML & NLP
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

# Resampling
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# NLTK stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/IMDB_Dataset.csv'
df = pd.read_csv('IMDB_Dataset.csv')
df.head()

In [ ]:
# Hapus nilai kosong
df.dropna(inplace=True)

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)          # hapus HTML tag
    text = re.sub(r'[^a-zA-Z]', ' ', text)     # hapus angka & simbol
    text = text.lower()                        # ubah ke huruf kecil
    text = text.split()                        # tokenisasi sederhana
    text = [word for word in text if word not in stop_words]  # hapus stopwords
    return ' '.join(text)

df.dropna(inplace=True)
df['clean_review'] = df['review'].apply(clean_text)
df['label'] = df['sentiment'].map({'negative':0, 'positive':1})
df = df[['clean_review', 'label']].dropna()
df.head()


In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_review'], df['label'],
    test_size=0.2, random_state=42, stratify=df['label']
)

# Vectorizer & to_dense untuk SMOTE
tfidf = TfidfVectorizer(max_features=5000, min_df=2)
to_array = FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)



In [ ]:
# Pipeline SMOTE untuk 2 model
pipelines = {
    "NaiveBayes_SMOTE": ImbPipeline(steps=[
        ("tfidf", tfidf),
        ("to_array", to_array),
        ("smote", SMOTE(random_state=42, k_neighbors=5)),
        ("clf", MultinomialNB())
    ]),
    "LogReg_SMOTE": ImbPipeline(steps=[
        ("tfidf", tfidf),
        ("to_array", to_array),
        ("smote", SMOTE(random_state=42, k_neighbors=5)),
        ("clf", LogisticRegression(max_iter=400))
    ])
}

In [ ]:
# Training, Prediksi, Evaluasi
rows = []
cms = {}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, pred, average='binary', pos_label=1, zero_division=0
    )
    cm = confusion_matrix(y_test, pred)

    rows.append([name, acc, prec, rec, f1])
    cms[name] = cm

results = pd.DataFrame(rows, columns=["Model", "Accuracy", "Precision", "Recall", "F1"]).sort_values("F1", ascending=False)
print(results.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

In [ ]:
# Classification Report detail
for name, pipe in pipelines.items():
    pred = pipe.predict(X_test)
    print(f"\n=== Classification Report: {name} ===")
    print(classification_report(y_test, pred, digits=4))

In [ ]:
# Plot Confusion Matrix untuk Kedua Model
fig, axes = plt.subplots(1, 2, figsize=(10,4))
for ax, (name, cm) in zip(axes, cms.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(f"Confusion Matrix - {name}")
    ax.set_xlabel("Prediksi")
    ax.set_ylabel("Aktual")
plt.tight_layout()
plt.show()


In [ ]:
best = results.iloc[0]
print(f"Model terbaik (berdasarkan F1): {best['Model']} | Acc={best['Accuracy']:.4f}, Prec={best['Precision']:.4f}, Rec={best['Recall']:.4f}, F1={best['F1']:.4f}")

In [ ]:
wc_kwargs = dict(
    width=1200,
    height=600,
    background_color="white",
    stopwords=stop_words,
    collocations=False
)

# Ambil model terbaik dan prediksi
best_name = results.sort_values("F1", ascending=False).iloc[0]["Model"]
best_pipe = pipelines[best_name]
y_pred = best_pipe.predict(X_test)

# Buat DataFrame untuk analisis prediksi
test_df = pd.DataFrame({
    "text": X_test.values,
    "y_true": y_test.values,
    "y_pred": y_pred
})

# Gabungkan teks berdasarkan hasil prediksi
pos_pred_text = " ".join(test_df.loc[test_df["y_pred"] == 1, "text"].astype(str).values)
neg_pred_text = " ".join(test_df.loc[test_df["y_pred"] == 0, "text"].astype(str).values)

# Tampilkan WordCloud
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.imshow(WordCloud(**wc_kwargs).generate(pos_pred_text))
plt.title(f"WordCloud — Prediksi Positif ({best_name})")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(WordCloud(**wc_kwargs).generate(neg_pred_text))
plt.title(f"WordCloud — Prediksi Negatif ({best_name})")
plt.axis("off")

plt.tight_layout()
plt.show()
